In [91]:
# Class for accessing financial data
import yfinance as yf

# Class(s) for Data analysis and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Class for  Connecting to Interactive Brokers
from ib_insync import *
import nest_asyncio

# Class for portfolio optimization
import riskfolio as rf

# Classes for Web scraping
import requests
from bs4 import BeautifulSoup

# Class(s) for supportive functions
from datetime import datetime as dt
import math
import random
import copy
from time import sleep
import warnings
from waiting import wait, TimeoutExpired

In [92]:
pd.options.display.float_format = "{:.4%}".format
warnings.filterwarnings("ignore")

In [93]:
url = "http://en.wikipedia.org/wiki/Nasdaq-100#Components"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    html_content = response.text  # Print the HTML content of the page
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    company_ticker = pd.read_html(str(table))[0]
    nasdaq_stocks = company_ticker['Ticker']
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching URL: {e}")

In [94]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    html_content = response.text  # Print the HTML content of the page
    soup = BeautifulSoup(html_content, 'html.parser')
    tables = pd.read_html(str(soup))
    sp_stocks = tables[0]['Symbol']
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching URL: {e}")

In [95]:
nasdaq_ticker_list = nasdaq_stocks.tolist()

In [96]:
sp_ticker_list = sp_stocks.tolist()

In [97]:
# Changing the ticker format to match the format in Yahoo finance for the following BRK.B and BF.B
sp_ticker_list[sp_ticker_list.index('BRK.B')] = 'BRK-B'
sp_ticker_list[sp_ticker_list.index('BF.B')] = 'BF-B'

In [98]:
# Combining the two lists and removing duplicates
long_ticker = list(set(sp_ticker_list + nasdaq_ticker_list))

In [99]:
long_df = pd.DataFrame()
failed_tickers = []
for ticker in long_ticker:
    try:
        long_df[ticker] = yf.download(ticker, period = '3y', interval = '1mo')['Close']
        # Calculate technical indicators
        # Using Simple Moving Average to determine Stocks that have upward momentum
        long_df[f'{ticker}_SMA5'] = long_df[ticker].rolling(window=5).mean()
        long_df[f'{ticker}_SMA15'] = long_df[ticker].rolling(window=15).mean()
        long_df[f'{ticker}_prev_SMA5'] = long_df[f'{ticker}_SMA5'].shift(1)
        long_df[f'{ticker}_prev_SMA15'] = long_df[f'{ticker}_SMA15'].shift(1)

    except Exception as e:
        failed_tickers.append(ticker)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [100]:
bullish_list=[]
bearish_list=[]
error_list=[]
for ticker in long_ticker:
    try:
        condition = (long_df[f'{ticker}_SMA5'] > long_df[f'{ticker}_SMA15']) & (long_df[f'{ticker}_prev_SMA5'] >= long_df[f'{ticker}_prev_SMA15'])
        if condition.iloc[-1] == True:
            bullish_list.append(ticker)
        else:
            bearish_list.append(ticker)
    except Exception as e:
        error_list.append((ticker, str(e)))

bullish_list

['WFC',
 'O',
 'NOC',
 'CDNS',
 'HCA',
 'CHRW',
 'TMUS',
 'ORLY',
 'CVS',
 'LHX',
 'CCI',
 'NFLX',
 'URI',
 'TJX',
 'TRI',
 'PAYC',
 'PGR',
 'MMM',
 'CMS',
 'BA',
 'LRCX',
 'GILD',
 'EW',
 'V',
 'SPG',
 'BK',
 'CFG',
 'COF',
 'AJG',
 'DPZ',
 'SMCI',
 'KR',
 'PM',
 'PAYX',
 'MNST',
 'SYY',
 'DTE',
 'GPC',
 'DDOG',
 'GOOGL',
 'FTNT',
 'GOOG',
 'BKNG',
 'EA',
 'YUM',
 'WAB',
 'UBER',
 'WEC',
 'NDAQ',
 'XEL',
 'T',
 'LUV',
 'EVRG',
 'AMT',
 'C',
 'CCEP',
 'NSC',
 'GEV',
 'F',
 'ROL',
 'BSX',
 'MO',
 'FOXA',
 'TDY',
 'RTX',
 'MPC',
 'SOLV',
 'CF',
 'CRWD',
 'RF',
 'VZ',
 'COR',
 'SNA',
 'ETN',
 'NEM',
 'WM',
 'PNW',
 'J',
 'WDC',
 'WMB',
 'GLW',
 'DXCM',
 'STT',
 'ETR',
 'APH',
 'JBL',
 'PH',
 'HII',
 'DLTR',
 'PODD',
 'PTC',
 'BRK-B',
 'DE',
 'GS',
 'HON',
 'LH',
 'ED',
 'EXPE',
 'DUK',
 'EXC',
 'LIN',
 'LDOS',
 'TKO',
 'BKR',
 'WTW',
 'MCD',
 'EMR',
 'CSCO',
 'AIG',
 'XYL',
 'NWS',
 'BLK',
 'SYF',
 'WELL',
 'INCY',
 'BR',
 'PEG',
 'AEE',
 'LYV',
 'TEL',
 'KMI',
 'ALL',
 'DRI',
 'HWM',
 'C

In [101]:
bearish_list

['XYZ',
 'EQIX',
 'CLX',
 'MPWR',
 'ISRG',
 'IQV',
 'AOS',
 'A',
 'AES',
 'NVR',
 'PCG',
 'EPAM',
 'CI',
 'PCAR',
 'KIM',
 'EIX',
 'TMO',
 'KHC',
 'TRGP',
 'PFG',
 'TER',
 'GIS',
 'EL',
 'AMAT',
 'NDSN',
 'CRL',
 'ADBE',
 'MRK',
 'FTV',
 'BRO',
 'PRU',
 'MET',
 'PHM',
 'FDX',
 'CVX',
 'IRM',
 'ERIE',
 'DECK',
 'GNRC',
 'REGN',
 'DOV',
 'TPL',
 'MOH',
 'VLTO',
 'CHD',
 'CB',
 'POOL',
 'BMY',
 'SWKS',
 'AON',
 'PSX',
 'LULU',
 'BG',
 'PLD',
 'MSCI',
 'ACGL',
 'IT',
 'XOM',
 'CARR',
 'AKAM',
 'LEN',
 'CPAY',
 'LII',
 'ELV',
 'PEP',
 'CMG',
 'ROST',
 'AAPL',
 'GDDY',
 'KMX',
 'HD',
 'MTD',
 'EQR',
 'CPT',
 'WY',
 'WST',
 'ARM',
 'JBHT',
 'LOW',
 'UPS',
 'STZ',
 'EFX',
 'GPN',
 'INTC',
 'MHK',
 'DVN',
 'CPB',
 'DD',
 'ARE',
 'OTIS',
 'GWW',
 'ON',
 'SRE',
 'BAX',
 'LLY',
 'NTAP',
 'MRNA',
 'NKE',
 'FICO',
 'AMCR',
 'NUE',
 'DHR',
 'SBAC',
 'TYL',
 'CHTR',
 'ADM',
 'SW',
 'TAP',
 'QCOM',
 'CTSH',
 'FIS',
 'NXPI',
 'COP',
 'HPQ',
 'TXN',
 'GFS',
 'APA',
 'CTRA',
 'COO',
 'HAL',
 'MDLZ',
 'MAS

In [102]:
print(len(bullish_list))
print(len(bearish_list))

272
245


In [103]:
# Calculating the performance KPIs

# Calculating the Compounded Annual Growth Rate
def CAGR(DF):
    df = DF.copy()
    df['cum_return'] = (1 + df['mon_return']).cumprod()
    n = len(df)/12 # the denominator is the number of trading periods in a year. In this case, 12 months
    CAGR = (df['cum_return'].tolist()[-1]) ** (1/n)-1
    return CAGR

# Calculating the Volatility
def volatility(DF):
    df = DF.copy()
    vol  = df['mon_return'].std() * np.sqrt(12) # The number in the sqrt is the number of r trading periods in a year
    return vol

# Calculating the Sharpe Ratio
def sharpe(DF, rf):
    df = DF.copy()
    sharpe = (CAGR(df) - .03) / volatility(df)
    return sharpe

# Calulating the maximum drawdown
def max_dd(DF):
    df = DF.copy()
    df['cum_return'] = (1 + df['mon_return']).cumprod()
    df['cum_roll_max'] = df['cum_return'].cummax()
    df['drawdown'] = df['cum_roll_max'] - df['cum_return']
    df['drawdown_pct'] = df['drawdown'] / df['cum_roll_max']
    max_dd = df['drawdown_pct'].max()
    return max_dd

In [104]:
def pflio(DF,m,x):
    """Returns cumulative portfolio return
    DF = dataframe with monthly return info for all stocks
    m = number of stocks to keep in the portfolio
    x = number of underperforming stocks to be removed from portfolio monthly"""
    df = DF.copy()
    portfolio = []
    monthly_ret = [0]
    for i in range(len(df)):
        if len(portfolio) > 0:
            monthly_ret.append(df[portfolio].iloc[i,:].mean())
            bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=True)[:x].index.values.tolist()
            portfolio = [t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_picks = df.iloc[i,:].sort_values(ascending=False)[:fill].index.values.tolist()
        portfolio = portfolio + new_picks
        print('\n list of stocks to go long: \n', portfolio)
    return portfolio

In [105]:
# Creating an empty dictionary which will be filled with Open, High, Low, Close, Volume dataframe for each ticker
ohlcv_data = {} 
for ticker in bullish_list: 
    ohlcv_data[ticker] = yf.download(ticker, period = '5y', interval = '1mo')
    ohlcv_data[ticker].dropna(inplace = True, how = 'all')
bullish_tickers = ohlcv_data.keys()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [106]:
ohlcv_dict = copy.deepcopy(ohlcv_data)
bullish_return_df = pd.DataFrame()
for ticker in bullish_tickers:
    print('calculating monthly return for',ticker)
    ohlcv_dict[ticker]['mon_return'] = ohlcv_dict[ticker]['Close'].pct_change()
    bullish_return_df[ticker] = ohlcv_dict[ticker]['mon_return']
bullish_return_df.dropna(how='all',axis=0, inplace=True)

calculating monthly return for WFC
calculating monthly return for O
calculating monthly return for NOC
calculating monthly return for CDNS
calculating monthly return for HCA
calculating monthly return for CHRW
calculating monthly return for TMUS
calculating monthly return for ORLY
calculating monthly return for CVS
calculating monthly return for LHX
calculating monthly return for CCI
calculating monthly return for NFLX
calculating monthly return for URI
calculating monthly return for TJX
calculating monthly return for TRI
calculating monthly return for PAYC
calculating monthly return for PGR
calculating monthly return for MMM
calculating monthly return for CMS
calculating monthly return for BA
calculating monthly return for LRCX
calculating monthly return for GILD
calculating monthly return for EW
calculating monthly return for V
calculating monthly return for SPG
calculating monthly return for BK
calculating monthly return for CFG
calculating monthly return for COF
calculating monthly

In [107]:
long_list = pflio(bullish_return_df,20,10)


 list of stocks to go long: 
 ['PLTR', 'MSTR', 'UBER', 'TSLA', 'BA', 'CCL', 'RCL', 'WYNN', 'GE', 'NWS', 'MAR', 'HWM', 'CF', 'C', 'LYV', 'NWSA', 'SPG', 'ULTA', 'UAL', 'WBD']

 list of stocks to go long: 
 ['MSTR', 'TSLA', 'CCL', 'WYNN', 'GE', 'HWM', 'CF', 'C', 'LYV', 'WBD', 'CRWD', 'ZS', 'TSLA', 'WDC', 'DIS', 'HWM', 'RL', 'PANW', 'FTNT', 'CDNS']

 list of stocks to go long: 
 ['MSTR', 'TSLA', 'GE', 'CF', 'WBD', 'CRWD', 'ZS', 'TSLA', 'WDC', 'PANW', 'MSTR', 'PLTR', 'WBD', 'AXON', 'PSKY', 'EQT', 'WSM', 'GM', 'F', 'IVZ']

 list of stocks to go long: 
 ['MSTR', 'GE', 'CF', 'WBD', 'WDC', 'MSTR', 'WBD', 'PSKY', 'EQT', 'F', 'RCL', 'CCL', 'LYV', 'TPR', 'PSKY', 'WYNN', 'LUV', 'UAL', 'LVS', 'EXPE']

 list of stocks to go long: 
 ['GE', 'CF', 'WDC', 'EQT', 'F', 'CCL', 'TPR', 'LUV', 'UAL', 'EXPE', 'WSM', 'HIG', 'STLD', 'AZO', 'JBL', 'BA', 'SMCI', 'CAH', 'HII', 'ATO']

 list of stocks to go long: 
 ['CF', 'WDC', 'EQT', 'CCL', 'TPR', 'LUV', 'STLD', 'AZO', 'HII', 'ATO', 'STX', 'COF', 'AJG', 'GOOG', 'T

In [108]:
# Sleeping 1 minute since over 500 calls was made to yfinance
sleep(60)
port_returns = (
    yf.download(
        long_list,
        period  = '1mo' 
    )["Close"]
    .pct_change()
    .dropna()
)

[*********************100%***********************]  16 of 16 completed


In [109]:
# These factors is a list of indicies that aims to track the performance
# of a particular startegy applied to the U.S. equity market

factors = ["MTUM", "QUAL", "VLUE", "SIZE", "USMV"]

In [110]:
factor_returns = (
    yf.download(
        factors, 
        period  = '1mo'
    )["Close"]
    .pct_change()
    .dropna()
)

[*********************100%***********************]  5 of 5 completed


In [111]:
port = rf.Portfolio(returns = port_returns)

port.assets_stats(method_mu = "hist", 
                  method_cov = "ledoit"
                  )

port.lowerret = 0.00056488 * 1.5

loadings = rf.loadings_matrix(
    X = factor_returns,
    Y = port_returns, 
    feature_selection = "PCR",
    n_components = 0.95
)

In [112]:
loadings.style.format("{:.4f}").background_gradient(cmap='RdYlGn')

,const,MTUM,QUAL,SIZE,USMV,VLUE
APTV,0.0057,-0.0385,0.5372,1.0332,-0.9795,1.3115
AVGO,0.0056,0.7280,0.2678,0.0037,-2.4467,0.6152
CVS,0.0073,-0.2239,-0.0781,0.0529,0.3988,-0.0561
DXCM,0.0007,0.8703,0.8317,0.0911,2.5883,-0.3435
EBAY,-0.0009,-0.2487,0.1846,0.4242,0.9716,0.2172
GLW,0.0052,0.7372,0.2637,-0.1738,-1.2364,0.1687
GOOG,0.0080,0.7394,0.3639,-0.1084,-0.5365,0.0904
GOOGL,0.0081,0.7500,0.3639,-0.1116,-0.5940,0.1004
MU,0.0056,1.0352,0.5132,0.2130,-3.4420,1.0897
NEM,0.0073,0.2853,-0.0552,-0.3454,-0.2860,-0.2739


In [113]:
port.factors = factor_returns

port.factors_stats(
    method_mu = "hist",
    method_cov = "ledoit",
)

In [114]:
w = port.optimization(
    model = "FM",
    rm = "MV",
    obj = "Sharpe",
    hist = True
)

In [115]:
w.reset_index(inplace = True)
w.rename(columns={'index':'ticker', 'weights':'weights'}, inplace = True)
w.sort_values('weights')

,ticker,weights
15,WSM,0.0000%
3,DXCM,0.0000%
4,EBAY,0.9194%
8,MU,1.2731%
1,AVGO,1.9166%
11,RL,2.4434%
13,UAL,2.4985%
5,GLW,3.9011%
14,WDC,6.0893%
12,STX,6.9954%


In [116]:
w[w['weights'] < .01] = np.nan
w.dropna(inplace = True)
w


,ticker,weights
0,APTV,9.6365%
1,AVGO,1.9166%
2,CVS,19.3104%
5,GLW,3.9011%
6,GOOG,7.1737%
7,GOOGL,7.1072%
8,MU,1.2731%
9,NEM,16.3229%
10,PODD,14.4124%
11,RL,2.4434%


In [117]:
# Sorting the weights in descending order
w.sort_values('weights', ascending=False)

,ticker,weights
2,CVS,19.3104%
9,NEM,16.3229%
10,PODD,14.4124%
0,APTV,9.6365%
6,GOOG,7.1737%
7,GOOGL,7.1072%
12,STX,6.9954%
14,WDC,6.0893%
5,GLW,3.9011%
13,UAL,2.4985%


In [118]:
long_dict = {} # empty dictionary which will be filled with ohlcv dataframe for each ticker
op_ticker_list = w['ticker'].to_list()
for long_op_ticker in op_ticker_list:
    long_dict[long_op_ticker] = yf.download(long_op_ticker, period = '1mo')
    long_dict[long_op_ticker].dropna(inplace = True, how = 'all')
tickers = long_dict.keys()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [119]:
nest_asyncio.apply()

# Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 4002, clientId=random.randint(1, 99))

# Connect to IB TWS
# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId = random.randint(1,99))

<IB connected to 127.0.0.1:4002 clientId=57>

In [120]:
def buy_stock(long_ticker: str, buy_diff: float = None):
        
        # Checking the account balance
        acct_bal = float(ib.accountSummary()[9].value)
        
        stock = Stock(
            symbol = long_ticker, 
            exchange = 'SMART', 
            currency = 'USD'
        )
        
        purchase_amount = ((float(w[w['ticker'] == f'{long_ticker}']['weights']) * acct_bal)) / (long_dict[long_ticker]['Close'].iloc[-1])

        if buy_diff is None:
            action = Order(
                action = 'BUY', 
                totalQuantity = round(purchase_amount.item()), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )
        
        else:
            action = Order(
                action = 'BUY', 
                totalQuantity = math.ceil(buy_diff), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )  
        
        order = ib.placeOrder(stock, action)

In [121]:
def sell_stock(ticker: str, sell_diff: float = None):
        stock = Stock(
            symbol = ticker, 
            exchange = 'SMART', 
            currency = 'USD'
        )
        
        for i in range (len(ib.positions())):
            if ib.positions()[i].contract.symbol == ticker:
                sell_amount = ib.positions()[i].position

        if sell_diff is None:
            action = Order(
                action = 'SELL', 
                totalQuantity = sell_amount, 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )
        else:
            action = Order(
                action = 'SELL', 
                totalQuantity = math.ceil(sell_diff), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            ) 
        
        order = ib.placeOrder(stock, action)

In [122]:
current_holdings = []

for i in range(len(ib.positions())):
    current_holdings.append(ib.positions()[i].contract.symbol)

In [123]:
current_holdings = []

for i in range(len(ib.positions())):
    current_holdings.append(ib.positions()[i].contract.symbol)

for holdings in current_holdings:
    if holdings not in long_list: 
        try:
            sell_stock(holdings)
        except TimeoutExpired as timeout:
            print(f'Sale of {holdings} {timeout} ')
    
    elif holdings in long_list:
        for i in range(len(ib.positions())):
            if ib.positions()[i].contract.symbol == holdings:
                current_holding_port_percentage = ib.positions()[i].position * ib.positions()[i].avgCost / float(ib.accountSummary()[24].value)
                if not w[w['ticker'] == f'{holdings}'].empty:
                    new_holding_port_percentage = w[w['ticker'] == f'{holdings}']['weights'].iloc[0]
                    
                    if current_holding_port_percentage > new_holding_port_percentage:
                        sell_diff = current_holding_port_percentage - new_holding_port_percentage
                        try:
                            sell_stock(ticker = holdings, sell_diff = sell_diff)
                        except TimeoutExpired as timeout:
                            print(f'Sale of {holdings} has been after {timeout}')
                    
                    elif current_holding_port_percentage < new_holding_port_percentage:
                        buy_diff = new_holding_port_percentage - current_holding_port_percentage
                        try:
                            buy_stock(long_ticker = holdings, buy_diff = buy_diff)
                        except ValueError as e:
                            print(f'Error buying {holdings}: {e}')
                else:
                    sell_stock(holdings)
                    print(f'{holdings} not found in the weights DataFrame, Therefore {holdings} was sold')
    

In [124]:
for long_op_ticker in op_ticker_list:
    if long_op_ticker not in current_holdings:
        try:
            buy_stock(long_op_ticker)
            ib.sleep(15)
        except ValueError as e:
            print(f'Error buying stock {long_op_ticker}: {e}')

In [125]:
# Get current positions
positions = ib.positions()

# Print the positions
for position in positions:
    print(f"Symbol: {position.contract.symbol}, Quantity: {position.position}")


Symbol: GLW, Quantity: 1077.0
Symbol: APTV, Quantity: 981.0
Symbol: AVGO, Quantity: 50.0
Symbol: CVS, Quantity: 2253.0
Symbol: GOOG, Quantity: 249.0
Symbol: GOOGL, Quantity: 246.0
Symbol: MU, Quantity: 79.0
Symbol: NEM, Quantity: 1699.0
Symbol: PODD, Quantity: 319.0
Symbol: RL, Quantity: 58.0
Symbol: STX, Quantity: 277.0
Symbol: UAL, Quantity: 169.0
Symbol: WDC, Quantity: 474.0


In [126]:
# Loop through your positions and creating trailing stop orders
for position in positions:
    contract = position.contract
    quantity = position.position
    
    if quantity > 0:  # Only create trailing stop orders for long positions
        # Define the trailing stop order
        trailing_stop_order = Order(
            action = 'SELL',  # Sell to exit the position
            orderType = 'TRAIL',
            totalQuantity = abs(quantity),
            trailingPercent = 10.0,  # Adjust the trailing percentage as needed
            tif = 'GTC'
        )

        # Submit the order
        trade = ib.placeOrder(contract, trailing_stop_order)
        print(f"Trailing stop order placed for {contract.symbol}, to sell a Quantity of {quantity}")

Trailing stop order placed for GLW, to sell a Quantity of 1077.0
Trailing stop order placed for APTV, to sell a Quantity of 981.0
Trailing stop order placed for AVGO, to sell a Quantity of 50.0
Trailing stop order placed for CVS, to sell a Quantity of 2253.0
Trailing stop order placed for GOOG, to sell a Quantity of 249.0
Trailing stop order placed for GOOGL, to sell a Quantity of 246.0
Trailing stop order placed for MU, to sell a Quantity of 79.0
Trailing stop order placed for NEM, to sell a Quantity of 1699.0
Trailing stop order placed for PODD, to sell a Quantity of 319.0
Trailing stop order placed for RL, to sell a Quantity of 58.0
Trailing stop order placed for STX, to sell a Quantity of 277.0
Trailing stop order placed for UAL, to sell a Quantity of 169.0
Trailing stop order placed for WDC, to sell a Quantity of 474.0


In [127]:
# Fetch open orders
open_orders = ib.openOrders()
for open_order in open_orders:
    print(f"Open Orders: {open_orders}")

ib.disconnect()

Open Orders: [Order(orderId=24, clientId=57, action='SELL', totalQuantity=1077.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=25, clientId=57, action='SELL', totalQuantity=981.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=26, clientId=57, action='SELL', totalQuantity=50.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=27, clientId=57, action='SELL', totalQuantity=2253.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=28, clientId=57, action='SELL', totalQuantity=249.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=29, clientId=57, action='SELL', totalQuantity=246.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=30, clientId=57, action='SELL', totalQuantity=79.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=31, clientId=57, action='SELL', totalQuantity=1699.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=32, clientI